# Importing Packages

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from arch import arch_model #GARCH Models
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import numpy as np
import math
import scipy.stats
from statistics import NormalDist
from scipy.stats import t
from scipy.stats import f
from datetime import datetime
from scipy.stats import shapiro #Shapiro Test
from scipy import stats #t-test
import statsmodels.api as sm #Ljung-Box Test
from scipy.stats import chi2

# Price and Returns

In [3]:
start_date = '2006-01-01' #yyyy-mm-dd
end_date = '2021-01-01'


#Philippines
PH = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/2006-2021/PSEi.csv')
PH['Date'] = pd.to_datetime(PH['Date'])
PH = PH.rename(columns={'Price': 'Close'})
PH = PH.replace(',','', regex=True)
PH['Close'] = PH['Close'].astype(float, errors = 'raise')
mask = (PH['Date'] >= start_date) & (PH['Date'] <= end_date)
PH = PH.loc[mask]
PH = PH.set_index('Date')
PH = PH.sort_index(axis=0, ascending = False)

In [4]:
def get_returns(df,d):
    '''
    The function obtains the log returns of the asset shifted d days
    
    PARAMETERS
    ----------
    df : pandas.DataFrame
        The data frame contains data of a chosen stock index 
        (Stock index must be arrange in DESCENDING ORDER by DATE)
    d : int
        The dth day being forecast
        (Assumed to be 1 for most cases)
        
    RETURNS
    -------
    df : pandas.DataFrame
        The data frame returns an updated data frame containing the
        'Returns' column
    '''
    df['Previous'] = df['Close'].shift(-d)
    df['Returns'] = np.log(df['Close']/df['Previous'])*100
    display(df)
    return df

PH = get_returns(PH, 1)

,Open,High,Low,Close,Unnamed: 5,Unnamed: 6,Previous,Returns
Date,,,,,,,,
2020-12-29,7123.12,7173.27,7120.82,7139.71,NaN,NaN,7122.25,0.244847
2020-12-28,7142.95,7142.95,7058.66,7122.25,NaN,NaN,7204.38,-1.146549
2020-12-23,7169.05,7206.46,7075.18,7204.38,NaN,NaN,7202.39,0.027626
2020-12-22,7206.33,7206.54,7081.14,7202.39,NaN,NaN,7224.89,-0.311909
2020-12-21,7244.85,7251.02,7157.88,7224.89,NaN,NaN,7272.80,-0.660935
...,...,...,...,...,...,...,...,...
2006-01-06,2150.57,2157.18,2132.05,2133.79,NaN,NaN,2150.93,-0.800057
2006-01-05,2138.29,2150.93,2129.34,2150.93,NaN,NaN,2138.05,0.600611
2006-01-04,2129.97,2144.04,2129.97,2138.05,NaN,NaN,2109.79,1.330578


# Diebold-Mariano Test

In [5]:
# Author   : John Tsang
# Date     : December 7th, 2017
# Purpose  : Implement the Diebold-Mariano Test (DM test) to compare 
#            forecast accuracy
# Input    : 1) actual_lst: the list of actual values
#            2) pred1_lst : the first list of predicted values
#            3) pred2_lst : the second list of predicted values
#            4) h         : the number of stpes ahead
#            5) crit      : a string specifying the criterion 
#                             i)  MSE : the mean squared error
#                            ii)  MAD : the mean absolute deviation
#                           iii) MAPE : the mean absolute percentage error
#                            iv) poly : use power function to weigh the errors
#            6) poly      : the power for crit power 
#                           (it is only meaningful when crit is "poly")
# Condition: 1) length of actual_lst, pred1_lst and pred2_lst is equal
#            2) h must be an integer and it must be greater than 0 and less than 
#               the length of actual_lst.
#            3) crit must take the 4 values specified in Input
#            4) Each value of actual_lst, pred1_lst and pred2_lst must
#               be numerical values. Missing values will not be accepted.
#            5) power must be a numerical value.
# Return   : a named-tuple of 2 elements
#            1) p_value : the p-value of the DM test
#            2) DM      : the test statistics of the DM test
##########################################################
# References:
#
# Harvey, D., Leybourne, S., & Newbold, P. (1997). Testing the equality of 
#   prediction mean squared errors. International Journal of forecasting, 
#   13(2), 281-291.
#
# Diebold, F. X. and Mariano, R. S. (1995), Comparing predictive accuracy, 
#   Journal of business & economic statistics 13(3), 253-264.
#
##########################################################
def dm_test(actual_lst, pred1_lst, pred2_lst, h = 1, crit="MSE", power = 2):
    # Routine for checking errors
    def error_check():
        rt = 0
        msg = ""
        # Check if h is an integer
        if (not isinstance(h, int)):
            rt = -1
            msg = "The type of the number of steps ahead (h) is not an integer."
            return (rt,msg)
        # Check the range of h
        if (h < 1):
            rt = -1
            msg = "The number of steps ahead (h) is not large enough."
            return (rt,msg)
        len_act = len(actual_lst)
        len_p1  = len(pred1_lst)
        len_p2  = len(pred2_lst)
        # Check if lengths of actual values and predicted values are equal
        if (len_act != len_p1 or len_p1 != len_p2 or len_act != len_p2):
            rt = -1
            msg = "Lengths of actual_lst, pred1_lst and pred2_lst do not match."
            return (rt,msg)
        # Check range of h
        if (h >= len_act):
            rt = -1
            msg = "The number of steps ahead is too large."
            return (rt,msg)
        # Check if criterion supported
        if (crit != "MSE" and crit != "MAPE" and crit != "MAD" and crit != "poly"):
            rt = -1
            msg = "The criterion is not supported."
            return (rt,msg)  
        # Check if every value of the input lists are numerical values
        from re import compile as re_compile
        comp = re_compile("^\d+?\.\d+?$")  
        def compiled_regex(s):
            """ Returns True is string is a number. """
            if comp.match(s) is None:
                return s.isdigit()
            return True
        for actual, pred1, pred2 in zip(actual_lst, pred1_lst, pred2_lst):
            is_actual_ok = compiled_regex(str(abs(actual)))
            is_pred1_ok = compiled_regex(str(abs(pred1)))
            is_pred2_ok = compiled_regex(str(abs(pred2)))
            if (not (is_actual_ok and is_pred1_ok and is_pred2_ok)):  
                msg = "An element in the actual_lst, pred1_lst or pred2_lst is not numeric."
                rt = -1
                return (rt,msg)
        return (rt,msg)
    
    # Error check
    error_code = error_check()
    # Raise error if cannot pass error check
    if (error_code[0] == -1):
        raise SyntaxError(error_code[1])
        return
    # Import libraries
    from scipy.stats import t
    import collections
    import pandas as pd
    import numpy as np
    
    # Initialise lists
    e1_lst = []
    e2_lst = []
    d_lst  = []
    
    # convert every value of the lists into real values
    actual_lst = pd.Series(actual_lst).apply(lambda x: float(x)).tolist()
    pred1_lst = pd.Series(pred1_lst).apply(lambda x: float(x)).tolist()
    pred2_lst = pd.Series(pred2_lst).apply(lambda x: float(x)).tolist()
    
    # Length of lists (as real numbers)
    T = float(len(actual_lst))
    
    # construct d according to crit
    if (crit == "MSE"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append((actual - p1)**2)
            e2_lst.append((actual - p2)**2)
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    elif (crit == "MAD"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(abs(actual - p1))
            e2_lst.append(abs(actual - p2))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    elif (crit == "MAPE"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(abs((actual - p1)/actual))
            e2_lst.append(abs((actual - p2)/actual))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    elif (crit == "poly"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(((actual - p1))**(power))
            e2_lst.append(((actual - p2))**(power))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)    
    
    # Mean of d        
    mean_d = pd.Series(d_lst).mean()
    
    # Find autocovariance and construct DM test statistics
    def autocovariance(Xi, N, k, Xs):
        autoCov = 0
        T = float(N)
        for i in np.arange(0, N-k):
              autoCov += ((Xi[i+k])-Xs)*(Xi[i]-Xs)
        return (1/(T))*autoCov
    gamma = []
    for lag in range(0,h):
        gamma.append(autocovariance(d_lst,len(d_lst),lag,mean_d)) # 0, 1, 2
    V_d = (gamma[0] + 2*sum(gamma[1:]))/T
    DM_stat=V_d**(-0.5)*mean_d
    harvey_adj=((T+1-2*h+h*(h-1)/T)/T)**(0.5)
    DM_stat = harvey_adj*DM_stat
    # Find p-value
    p_value = 2*t.cdf(-abs(DM_stat), df = T - 1)
    
    # Construct named tuple for return
    dm_return = collections.namedtuple('dm_return', 'DM p_value')
    
    rt = dm_return(DM = DM_stat, p_value = p_value)
    
    return rt

## Custom DM-Test Function

In [6]:
def DM_test(country, model_1, model_2, t):
    
    arr_model_1_dropped = model_1.drop(['Date'], axis = 1)
    arr_model_1_zero = arr_model_1_dropped.iloc[:, 0].to_numpy()

    arr_model_2_dropped = model_2.drop(['Date'], axis = 1)
    arr_model_2_zero = arr_model_2_dropped.iloc[:,0].to_numpy()

    returns = np.array(country['Returns'])[:t]

    dm = dm_test(returns, arr_model_1_zero, arr_model_2_zero, h = 1, crit = 'MSE')
    
    return dm

# Fixed Window

## 0.01%

In [7]:
PH_fw_01_dn = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/DN/0.01%25/PH_VaR_FW-DN-0.01%25.csv')
PH_fw_01_hs = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/HS/0.01%25/PH_VaR_FW-HS-0.01%25.csv')
PH_fw_01_brw = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/BRW/0.01%25/PH_VaR_FW-BRW-0.01%25.csv')
PH_fw_01_garch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/GARCH/0.01%25/PH_VaR_FW-G-0.01%25.csv')
PH_fw_01_earch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EARCH/0.01%25/PH_VaR_FW-EA-0.01%25.csv')
PH_fw_01_egarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EGARCH/0.01%25/PH_VaR_FW-E-0.01%25.csv')
PH_fw_01_igarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/IGARCH/IGARCH-0.01%25-FW/PH_fw_FW-I-0.01%25.csv')
PH_fw_01_figarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH/FIGARCH-0.01%25-FW/PH_fw_FW-FI-0.01%25.csv')
PH_fw_01_figarch0633 = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH0.633/FIGARCH0.633-0.01%25-FW/PH_fw_FW-FI0.633-0.01%25.csv')

In [8]:
# DN
print(DM_test(PH, PH_fw_01_dn, PH_fw_01_garch, len(PH_fw_01_dn)))
print(DM_test(PH, PH_fw_01_dn, PH_fw_01_earch, len(PH_fw_01_dn)))
print(DM_test(PH, PH_fw_01_dn, PH_fw_01_egarch, len(PH_fw_01_dn)))
print(DM_test(PH, PH_fw_01_dn, PH_fw_01_igarch, len(PH_fw_01_dn)))
print(DM_test(PH, PH_fw_01_dn, PH_fw_01_figarch, len(PH_fw_01_dn)))
print(DM_test(PH, PH_fw_01_dn, PH_fw_01_figarch0633, len(PH_fw_01_dn)))

dm_return(DM=-3.5160764065391885, p_value=0.000463269504543097)
dm_return(DM=-0.3407536255984439, p_value=0.7333808226343581)
dm_return(DM=-3.0027389360096537, p_value=0.002761079426311385)
dm_return(DM=-2.5225420090988746, p_value=0.011848865458636981)
dm_return(DM=-4.119268454373675, p_value=4.206856942093485e-05)
dm_return(DM=-3.478442168567482, p_value=0.0005322941057406609)


In [9]:
# HS
print(DM_test(PH, PH_fw_01_hs, PH_fw_01_garch, len(PH_fw_01_hs)))
print(DM_test(PH, PH_fw_01_hs, PH_fw_01_earch, len(PH_fw_01_hs)))
print(DM_test(PH, PH_fw_01_hs, PH_fw_01_egarch, len(PH_fw_01_hs)))
print(DM_test(PH, PH_fw_01_hs, PH_fw_01_igarch, len(PH_fw_01_hs)))
print(DM_test(PH, PH_fw_01_hs, PH_fw_01_figarch, len(PH_fw_01_hs)))
print(DM_test(PH, PH_fw_01_hs, PH_fw_01_figarch0633, len(PH_fw_01_hs)))

dm_return(DM=8.809822591543611, p_value=8.040640588118636e-18)
dm_return(DM=42.17542555544652, p_value=2.0864308095889964e-203)
dm_return(DM=15.527098773815593, p_value=1.4099730855283968e-47)
dm_return(DM=3.7545390157290184, p_value=0.00018656712799172535)
dm_return(DM=8.529485121656059, p_value=7.594157434321829e-17)
dm_return(DM=5.588212507324295, p_value=3.171532193637059e-08)


In [10]:
# BRW
print(DM_test(PH, PH_fw_01_brw, PH_fw_01_garch, len(PH_fw_01_brw)))
print(DM_test(PH, PH_fw_01_brw, PH_fw_01_earch, len(PH_fw_01_brw)))
print(DM_test(PH, PH_fw_01_brw, PH_fw_01_egarch, len(PH_fw_01_brw)))
print(DM_test(PH, PH_fw_01_brw, PH_fw_01_igarch, len(PH_fw_01_brw)))
print(DM_test(PH, PH_fw_01_brw, PH_fw_01_figarch, len(PH_fw_01_brw)))
print(DM_test(PH, PH_fw_01_brw, PH_fw_01_figarch0633, len(PH_fw_01_brw)))

dm_return(DM=1.5995394538399035, p_value=0.11010573741246787)
dm_return(DM=11.776154357683952, p_value=1.4011950808339368e-29)
dm_return(DM=4.446756791665255, p_value=9.982971774672081e-06)
dm_return(DM=0.10259653836027594, p_value=0.9183095603092271)
dm_return(DM=1.1561067189980616, p_value=0.24799148918751432)
dm_return(DM=0.31091426096091757, p_value=0.7559488709640833)


In [11]:
# GARCH
print(DM_test(PH, PH_fw_01_garch, PH_fw_01_earch, len(PH_fw_01_garch)))
print(DM_test(PH, PH_fw_01_garch, PH_fw_01_egarch, len(PH_fw_01_garch)))
print(DM_test(PH, PH_fw_01_garch, PH_fw_01_igarch, len(PH_fw_01_garch)))
print(DM_test(PH, PH_fw_01_garch, PH_fw_01_figarch, len(PH_fw_01_garch)))
print(DM_test(PH, PH_fw_01_garch, PH_fw_01_figarch0633, len(PH_fw_01_garch)))

dm_return(DM=3.7114946865788014, p_value=0.00022068395056166747)
dm_return(DM=4.312802569416899, p_value=1.8188061973717996e-05)
dm_return(DM=-1.0709251868673513, p_value=0.28453432717850363)
dm_return(DM=-2.902676742107884, p_value=0.0038042055066395174)
dm_return(DM=-2.09678692908013, p_value=0.03633393090568196)


In [12]:
# EARCH
print(DM_test(PH, PH_fw_01_earch, PH_fw_01_egarch, len(PH_fw_01_earch)))
print(DM_test(PH, PH_fw_01_earch, PH_fw_01_igarch, len(PH_fw_01_earch)))
print(DM_test(PH, PH_fw_01_earch, PH_fw_01_figarch, len(PH_fw_01_earch)))
print(DM_test(PH, PH_fw_01_earch, PH_fw_01_figarch0633, len(PH_fw_01_earch)))

dm_return(DM=-3.1739866393547196, p_value=0.0015626156520959545)
dm_return(DM=-2.6829202689723264, p_value=0.0074527947285733176)
dm_return(DM=-4.416379770545266, p_value=1.1453706535348804e-05)
dm_return(DM=-3.750285498250799, p_value=0.00018970312206328214)


In [13]:
# EGARCH
print(DM_test(PH, PH_fw_01_egarch, PH_fw_01_igarch, len(PH_fw_01_egarch)))
print(DM_test(PH, PH_fw_01_egarch, PH_fw_01_figarch, len(PH_fw_01_egarch)))
print(DM_test(PH, PH_fw_01_egarch, PH_fw_01_figarch0633, len(PH_fw_01_egarch)))

dm_return(DM=-1.9929662268173078, p_value=0.046613140199769046)
dm_return(DM=-5.983446314050501, p_value=3.327758248042429e-09)
dm_return(DM=-3.5262405924382056, p_value=0.0004461168621870455)


In [14]:
# IGARCH
print(DM_test(PH, PH_fw_01_igarch, PH_fw_01_figarch, len(PH_fw_01_igarch)))
print(DM_test(PH, PH_fw_01_igarch, PH_fw_01_figarch0633, len(PH_fw_01_igarch)))

dm_return(DM=0.7507478225646607, p_value=0.453030854345321)
dm_return(DM=0.285415742167007, p_value=0.7754014208364659)


In [15]:
# FIGARCH
print(DM_test(PH, PH_fw_01_figarch, PH_fw_01_figarch0633, len(PH_fw_01_figarch)))

dm_return(DM=-1.4622850431498933, p_value=0.14406589430164743)


## 1.00 %

In [16]:
PH_fw_1_dn = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/DN/1%25/PH_VaR_FW-DN-1%25.csv')
PH_fw_1_hs = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/HS/1%25/PH_VaR_FW-HS-1%25.csv')
PH_fw_1_brw = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/BRW/1%25/PH_VaR_FW-BRW-1%25.csv')
PH_fw_1_garch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/GARCH/1%25/PH_VaR_FW-G-1%25.csv')
PH_fw_1_earch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EARCH/1%25/PH_VaR_FW-EA-1%25.csv')
PH_fw_1_egarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EGARCH/1%25/PH_VaR_FW-E-1%25.csv')
PH_fw_1_igarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/IGARCH/IGARCH-1%25-FW/PH_fw_FW-I-1%25.csv')
PH_fw_1_figarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH/FIGARCH-1%25-FW/PH_fw_FW-FI-1%25.csv')
PH_fw_1_figarch0633 = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH0.633/FIGARCH0.633-1%25-FW/PH_fw_FW-FI0.633-1%25.csv')

In [17]:
# DN
print(DM_test(PH, PH_fw_1_dn, PH_fw_1_garch, len(PH_fw_1_dn)))
print(DM_test(PH, PH_fw_1_dn, PH_fw_1_earch, len(PH_fw_1_dn)))
print(DM_test(PH, PH_fw_1_dn, PH_fw_1_egarch, len(PH_fw_1_dn)))
print(DM_test(PH, PH_fw_1_dn, PH_fw_1_igarch, len(PH_fw_1_dn)))
print(DM_test(PH, PH_fw_1_dn, PH_fw_1_figarch, len(PH_fw_1_dn)))
print(DM_test(PH, PH_fw_1_dn, PH_fw_1_figarch0633, len(PH_fw_1_dn)))

dm_return(DM=-3.4762409419102585, p_value=0.0005366147356787624)
dm_return(DM=-0.3698274620918392, p_value=0.7116114396358482)
dm_return(DM=-2.9524231775836363, p_value=0.0032475299486857015)
dm_return(DM=-2.514414696307132, p_value=0.012123185950582779)
dm_return(DM=-4.072425941464794, p_value=5.127709659843171e-05)
dm_return(DM=-3.4423774298452146, p_value=0.0006073471416170175)


In [18]:
# HS
print(DM_test(PH, PH_fw_1_hs, PH_fw_1_garch, len(PH_fw_1_hs)))
print(DM_test(PH, PH_fw_1_hs, PH_fw_1_earch, len(PH_fw_1_hs)))
print(DM_test(PH, PH_fw_1_hs, PH_fw_1_egarch, len(PH_fw_1_hs)))
print(DM_test(PH, PH_fw_1_hs, PH_fw_1_igarch, len(PH_fw_1_hs)))
print(DM_test(PH, PH_fw_1_hs, PH_fw_1_figarch, len(PH_fw_1_hs)))
print(DM_test(PH, PH_fw_1_hs, PH_fw_1_figarch0633, len(PH_fw_1_hs)))

dm_return(DM=-3.11505186357847, p_value=0.0019064326075571953)
dm_return(DM=0.9647173174962824, p_value=0.3349854149972894)
dm_return(DM=-2.4126494046985583, p_value=0.016066778729425434)
dm_return(DM=-2.3374589072765852, p_value=0.019667378289817614)
dm_return(DM=-3.7006586785026925, p_value=0.00023015379725394884)
dm_return(DM=-3.181523495490357, p_value=0.0015230334969678699)


In [19]:
# BRW
print(DM_test(PH, PH_fw_1_brw, PH_fw_1_garch, len(PH_fw_1_brw)))
print(DM_test(PH, PH_fw_1_brw, PH_fw_1_earch, len(PH_fw_1_brw)))
print(DM_test(PH, PH_fw_1_brw, PH_fw_1_egarch, len(PH_fw_1_brw)))
print(DM_test(PH, PH_fw_1_brw, PH_fw_1_igarch, len(PH_fw_1_brw)))
print(DM_test(PH, PH_fw_1_brw, PH_fw_1_figarch, len(PH_fw_1_brw)))
print(DM_test(PH, PH_fw_1_brw, PH_fw_1_figarch0633, len(PH_fw_1_brw)))

dm_return(DM=2.543444740050071, p_value=0.011168507452243822)
dm_return(DM=13.541488230017201, p_value=1.124339368275917e-37)
dm_return(DM=5.812352203056531, p_value=8.976521117407938e-09)
dm_return(DM=0.5341017588309703, p_value=0.593423316814017)
dm_return(DM=2.102366849678673, p_value=0.035841133601464334)
dm_return(DM=0.9599403274462078, p_value=0.33738271165608935)


In [20]:
# GARCH
print(DM_test(PH, PH_fw_1_garch, PH_fw_1_earch, len(PH_fw_1_garch)))
print(DM_test(PH, PH_fw_1_garch, PH_fw_1_egarch, len(PH_fw_1_garch)))
print(DM_test(PH, PH_fw_1_garch, PH_fw_1_igarch, len(PH_fw_1_garch)))
print(DM_test(PH, PH_fw_1_garch, PH_fw_1_figarch, len(PH_fw_1_garch)))
print(DM_test(PH, PH_fw_1_garch, PH_fw_1_figarch0633, len(PH_fw_1_garch)))

dm_return(DM=3.665761628518185, p_value=0.0002633134997781572)
dm_return(DM=4.29769954756112, p_value=1.944133842780066e-05)
dm_return(DM=-1.117138702936723, p_value=0.2642790109892734)
dm_return(DM=-2.995084867829833, p_value=0.002830498233480178)
dm_return(DM=-2.1351544823619832, p_value=0.033059366494270555)


In [21]:
# EARCH
print(DM_test(PH, PH_fw_1_earch, PH_fw_1_egarch, len(PH_fw_1_earch)))
print(DM_test(PH, PH_fw_1_earch, PH_fw_1_igarch, len(PH_fw_1_earch)))
print(DM_test(PH, PH_fw_1_earch, PH_fw_1_figarch, len(PH_fw_1_earch)))
print(DM_test(PH, PH_fw_1_earch, PH_fw_1_figarch0633, len(PH_fw_1_earch)))

dm_return(DM=-3.1116957699442946, p_value=0.001927965218243067)
dm_return(DM=-2.669725802867528, p_value=0.007749252771569769)
dm_return(DM=-4.360345023856617, p_value=1.4726963119847188e-05)
dm_return(DM=-3.7050299634527017, p_value=0.00022628854606168217)


In [22]:
# EGARCH
print(DM_test(PH, PH_fw_1_egarch, PH_fw_1_igarch, len(PH_fw_1_egarch)))
print(DM_test(PH, PH_fw_1_egarch, PH_fw_1_figarch, len(PH_fw_1_egarch)))
print(DM_test(PH, PH_fw_1_egarch, PH_fw_1_figarch0633, len(PH_fw_1_egarch)))

dm_return(DM=-2.015330095063527, p_value=0.04421263933645943)
dm_return(DM=-5.9519701331689765, p_value=4.001705410621301e-09)
dm_return(DM=-3.5164531444732257, p_value=0.00046262288616328424)


In [23]:
# IGARCH
print(DM_test(PH, PH_fw_1_igarch, PH_fw_1_figarch, len(PH_fw_1_igarch)))
print(DM_test(PH, PH_fw_1_igarch, PH_fw_1_figarch0633, len(PH_fw_1_igarch)))

dm_return(DM=0.7905569702932714, p_value=0.42944285625005163)
dm_return(DM=0.33157117185800106, p_value=0.7403021568497206)


In [24]:
# FIGARCH
print(DM_test(PH, PH_fw_1_figarch, PH_fw_1_figarch0633, len(PH_fw_1_figarch)))

dm_return(DM=-1.485286904961992, p_value=0.13787186692250467)


## 5.00 %

In [25]:
PH_fw_5_dn = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/DN/5%25/PH_VaR_FW-DN-5%25.csv')
PH_fw_5_hs = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/HS/5%25/PH_VaR_FW-HS-5%25.csv')
PH_fw_5_brw = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/BRW/5%25/PH_VaR_FW-BRW-5%25.csv')
PH_fw_5_garch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/GARCH/5%25/PH_VaR_FW-G-5%25.csv')
PH_fw_5_earch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EARCH/5%25/PH_VaR_FW-EA-5%25.csv')
PH_fw_5_egarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EGARCH/5%25/PH_VaR_FW-E-5%25.csv')
PH_fw_5_igarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/IGARCH/IGARCH-5%25-FW/PH_fw_FW-I-5%25.csv')
PH_fw_5_figarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH/FIGARCH-5%25-FW/PH_fw_FW-FI-5%25.csv')
PH_fw_5_figarch0633 = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH0.633/FIGARCH0.633-5%25-FW/PH_fw_FW-FI0.633-5%25.csv')

In [26]:
# DN
print(DM_test(PH, PH_fw_5_dn, PH_fw_5_garch, len(PH_fw_5_dn)))
print(DM_test(PH, PH_fw_5_dn, PH_fw_5_earch, len(PH_fw_5_dn)))
print(DM_test(PH, PH_fw_5_dn, PH_fw_5_egarch, len(PH_fw_5_dn)))
print(DM_test(PH, PH_fw_5_dn, PH_fw_5_igarch, len(PH_fw_5_dn)))
print(DM_test(PH, PH_fw_5_dn, PH_fw_5_figarch, len(PH_fw_5_dn)))
print(DM_test(PH, PH_fw_5_dn, PH_fw_5_figarch0633, len(PH_fw_5_dn)))

dm_return(DM=-3.423655959579582, p_value=0.000650091775445083)
dm_return(DM=-0.3988592220129882, p_value=0.6901061056125186)
dm_return(DM=-2.8874986956250615, p_value=0.003990589422946735)
dm_return(DM=-2.5019417736409157, p_value=0.012555157085786429)
dm_return(DM=-4.01104063202717, p_value=6.626612020788282e-05)
dm_return(DM=-3.3968944644151535, p_value=0.0007160784851972559)


In [27]:
# HS
print(DM_test(PH, PH_fw_5_hs, PH_fw_5_garch, len(PH_fw_5_hs)))
print(DM_test(PH, PH_fw_5_hs, PH_fw_5_earch, len(PH_fw_5_hs)))
print(DM_test(PH, PH_fw_5_hs, PH_fw_5_egarch, len(PH_fw_5_hs)))
print(DM_test(PH, PH_fw_5_hs, PH_fw_5_igarch, len(PH_fw_5_hs)))
print(DM_test(PH, PH_fw_5_hs, PH_fw_5_figarch, len(PH_fw_5_hs)))
print(DM_test(PH, PH_fw_5_hs, PH_fw_5_figarch0633, len(PH_fw_5_hs)))

dm_return(DM=-5.5666177176890885, p_value=3.573502271134942e-08)
dm_return(DM=-8.673896305896406, p_value=2.4058833820542457e-17)
dm_return(DM=-6.089920209006612, p_value=1.772240742569298e-09)
dm_return(DM=-3.578071362544603, p_value=0.00036751097482096823)
dm_return(DM=-6.173905732444161, p_value=1.0709183127861763e-09)
dm_return(DM=-4.944196138764234, p_value=9.364919357531264e-07)


In [28]:
# BRW
print(DM_test(PH, PH_fw_5_brw, PH_fw_5_garch, len(PH_fw_5_brw)))
print(DM_test(PH, PH_fw_5_brw, PH_fw_5_earch, len(PH_fw_5_brw)))
print(DM_test(PH, PH_fw_5_brw, PH_fw_5_egarch, len(PH_fw_5_brw)))
print(DM_test(PH, PH_fw_5_brw, PH_fw_5_igarch, len(PH_fw_5_brw)))
print(DM_test(PH, PH_fw_5_brw, PH_fw_5_figarch, len(PH_fw_5_brw)))
print(DM_test(PH, PH_fw_5_brw, PH_fw_5_figarch0633, len(PH_fw_5_brw)))

dm_return(DM=-2.4503196065704143, p_value=0.01449161776122256)
dm_return(DM=2.759640870960774, p_value=0.005922189537626883)
dm_return(DM=-1.4388909484967076, p_value=0.15058256579768148)
dm_return(DM=-2.014944498845563, p_value=0.04425312550479678)
dm_return(DM=-3.0451729991684653, p_value=0.0024036975859183535)
dm_return(DM=-2.6951172058469317, p_value=0.0071878564862018395)


In [29]:
# GARCH
print(DM_test(PH, PH_fw_5_garch, PH_fw_5_earch, len(PH_fw_5_garch)))
print(DM_test(PH, PH_fw_5_garch, PH_fw_5_egarch, len(PH_fw_5_garch)))
print(DM_test(PH, PH_fw_5_garch, PH_fw_5_igarch, len(PH_fw_5_garch)))
print(DM_test(PH, PH_fw_5_garch, PH_fw_5_figarch, len(PH_fw_5_garch)))
print(DM_test(PH, PH_fw_5_garch, PH_fw_5_figarch0633, len(PH_fw_5_garch)))

dm_return(DM=3.607088174425817, p_value=0.000329372876043967)
dm_return(DM=4.274336619208395, p_value=2.1543671718988636e-05)
dm_return(DM=-1.1652405354207533, p_value=0.24427786033936186)
dm_return(DM=-3.0837172849896133, p_value=0.0021163751637797235)
dm_return(DM=-2.172505369723845, p_value=0.030118017097862174)


In [30]:
# EARCH
print(DM_test(PH, PH_fw_5_earch, PH_fw_5_egarch, len(PH_fw_5_earch)))
print(DM_test(PH, PH_fw_5_earch, PH_fw_5_igarch, len(PH_fw_5_earch)))
print(DM_test(PH, PH_fw_5_earch, PH_fw_5_figarch, len(PH_fw_5_earch)))
print(DM_test(PH, PH_fw_5_earch, PH_fw_5_figarch0633, len(PH_fw_5_earch)))

dm_return(DM=-3.034347171148277, p_value=0.002490602087255015)
dm_return(DM=-2.6527222273914477, p_value=0.008146890935735545)
dm_return(DM=-4.289493128538706, p_value=2.015644881805436e-05)
dm_return(DM=-3.650702580130906, p_value=0.00027896635376421336)


In [31]:
# EGARCH
print(DM_test(PH, PH_fw_5_egarch, PH_fw_5_igarch, len(PH_fw_5_egarch)))
print(DM_test(PH, PH_fw_5_egarch, PH_fw_5_figarch, len(PH_fw_5_egarch)))
print(DM_test(PH, PH_fw_5_egarch, PH_fw_5_figarch0633, len(PH_fw_5_egarch)))

dm_return(DM=-2.036772850593314, p_value=0.04201001574570609)
dm_return(DM=-5.905352096446973, p_value=5.250461398307117e-09)
dm_return(DM=-3.5011433576581883, p_value=0.0004895910570852146)


In [32]:
# IGARCH
print(DM_test(PH, PH_fw_5_igarch, PH_fw_5_figarch,  len(PH_fw_5_igarch)))
print(DM_test(PH, PH_fw_5_igarch, PH_fw_5_figarch0633, len(PH_fw_5_igarch)))

dm_return(DM=0.8321923372003982, p_value=0.4055550973476739)
dm_return(DM=0.3809373703977944, p_value=0.7033534503610996)


In [33]:
# FIGARCH
print(DM_test(PH, PH_fw_5_figarch, PH_fw_5_figarch0633, len(PH_fw_5_figarch)))

dm_return(DM=-1.5075108516450824, p_value=0.1320846593399229)


# Rolling Window

## 0.01 %

In [34]:
PH_rw_01_dn = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/DN/0.01%25/PH_VaR_RW-DN-0.01%25.csv')
PH_rw_01_hs = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/HS/0.01%25/PH_VaR_RW-HS-0.01%25.csv')
PH_rw_01_brw = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/BRW/0.01%25/PH_VaR_RW-BRW-0.01%25.csv')
PH_rw_01_garch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/GARCH/0.01%25/PH_VaR_RW-G-0.01%25.csv')
PH_rw_01_earch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EARCH/0.01%25/PH_VaR_RW-EA-0.01%25.csv')
PH_rw_01_egarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EGARCH/0.01%25/PH_VaR_RW-E-0.01%25.csv')
PH_rw_01_igarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/IGARCH/IGARCH-0.01%25-RW/PH_RW-I-0.01%25.csv')
PH_rw_01_figarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH/FIGARCH-0.01%25-RW/PH_RW-FI-0.01%25.csv')
PH_rw_01_figarch0633 = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH0.633/FIGARCH0.633-0.01%25-RW/PH_RW-FI0.633-0.01%25.csv')

In [35]:
# DN
print(DM_test(PH, PH_rw_01_dn, PH_rw_01_garch, len(PH_rw_01_dn)))
print(DM_test(PH, PH_rw_01_dn, PH_rw_01_earch, len(PH_rw_01_dn)))
print(DM_test(PH, PH_rw_01_dn, PH_rw_01_egarch, len(PH_rw_01_dn)))
print(DM_test(PH, PH_rw_01_dn, PH_rw_01_igarch, len(PH_rw_01_dn)))
print(DM_test(PH, PH_rw_01_dn, PH_rw_01_figarch, len(PH_rw_01_dn)))
print(DM_test(PH, PH_rw_01_dn, PH_rw_01_figarch0633, len(PH_rw_01_dn)))

dm_return(DM=-3.3641647006545217, p_value=0.0008052510765809866)
dm_return(DM=-0.35498972014751584, p_value=0.7226932557537511)
dm_return(DM=-2.706151309172739, p_value=0.006955506858656721)
dm_return(DM=-2.3471655082276484, p_value=0.01916600452162084)
dm_return(DM=-3.716458962359054, p_value=0.0002164690779010886)
dm_return(DM=-3.2547206623404303, p_value=0.001183977129116786)


In [36]:
# HS
print(DM_test(PH, PH_rw_01_hs, PH_rw_01_garch, len(PH_rw_01_hs)))
print(DM_test(PH, PH_rw_01_hs, PH_rw_01_earch, len(PH_rw_01_hs)))
print(DM_test(PH, PH_rw_01_hs, PH_rw_01_egarch, len(PH_rw_01_hs)))
print(DM_test(PH, PH_rw_01_hs, PH_rw_01_igarch, len(PH_rw_01_hs)))
print(DM_test(PH, PH_rw_01_hs, PH_rw_01_figarch, len(PH_rw_01_hs)))
print(DM_test(PH, PH_rw_01_hs, PH_rw_01_figarch0633, len(PH_rw_01_hs)))

dm_return(DM=26.513870029773148, p_value=6.176741520748985e-111)
dm_return(DM=87.10094609639425, p_value=0.0)
dm_return(DM=41.30182141581952, p_value=1.6580382805972348e-198)
dm_return(DM=13.53132039662655, p_value=1.257655923979145e-37)
dm_return(DM=26.738820654020913, p_value=2.6679118676292486e-112)
dm_return(DM=19.723152894143343, p_value=1.4534973192907487e-70)


In [37]:
# BRW
print(DM_test(PH, PH_rw_01_brw, PH_rw_01_garch, len(PH_rw_01_brw)))
print(DM_test(PH, PH_rw_01_brw, PH_rw_01_earch, len(PH_rw_01_brw)))
print(DM_test(PH, PH_rw_01_brw, PH_rw_01_egarch, len(PH_rw_01_brw)))
print(DM_test(PH, PH_rw_01_brw, PH_rw_01_igarch, len(PH_rw_01_brw)))
print(DM_test(PH, PH_rw_01_brw, PH_rw_01_figarch, len(PH_rw_01_brw)))
print(DM_test(PH, PH_rw_01_brw, PH_rw_01_figarch0633, len(PH_rw_01_brw)))

dm_return(DM=1.2271747722045663, p_value=0.22012716462562218)
dm_return(DM=10.676298357632016, p_value=6.276167842329172e-25)
dm_return(DM=4.005674256013925, p_value=6.775760857274536e-05)
dm_return(DM=0.063690029411571, p_value=0.9492333619066557)
dm_return(DM=0.9705940526816553, p_value=0.332051341962608)
dm_return(DM=0.25479871172764706, p_value=0.7989457332845002)


In [38]:
# GARCH
print(DM_test(PH, PH_rw_01_garch, PH_rw_01_earch, len(PH_rw_01_garch)))
print(DM_test(PH, PH_rw_01_garch, PH_rw_01_egarch, len(PH_rw_01_garch)))
print(DM_test(PH, PH_rw_01_garch, PH_rw_01_igarch, len(PH_rw_01_garch)))
print(DM_test(PH, PH_rw_01_garch, PH_rw_01_figarch, len(PH_rw_01_garch)))
print(DM_test(PH, PH_rw_01_garch, PH_rw_01_figarch0633, len(PH_rw_01_garch)))

dm_return(DM=3.537014960488928, p_value=0.00042858408543946315)
dm_return(DM=4.426454609881679, p_value=1.0944348364157837e-05)
dm_return(DM=-0.8661567794674584, p_value=0.38667053463208534)
dm_return(DM=-1.5384073860395282, p_value=0.12435458387482025)
dm_return(DM=-1.6764232538763213, p_value=0.09405603101325967)


In [39]:
# EARCH
print(DM_test(PH, PH_rw_01_earch, PH_rw_01_egarch, len(PH_rw_01_earch)))
print(DM_test(PH, PH_rw_01_earch, PH_rw_01_igarch, len(PH_rw_01_earch)))
print(DM_test(PH, PH_rw_01_earch, PH_rw_01_figarch, len(PH_rw_01_earch)))
print(DM_test(PH, PH_rw_01_earch, PH_rw_01_figarch0633, len(PH_rw_01_earch)))

dm_return(DM=-2.8240627999679377, p_value=0.0048628434420098215)
dm_return(DM=-2.5009502132415693, p_value=0.01259007637758689)
dm_return(DM=-3.996148408767567, p_value=7.048384897400452e-05)
dm_return(DM=-3.5196921490296975, p_value=0.00045709829107732974)


In [40]:
# EGARCH
print(DM_test(PH, PH_rw_01_egarch, PH_rw_01_igarch, len(PH_rw_01_egarch)))
print(DM_test(PH, PH_rw_01_egarch, PH_rw_01_figarch, len(PH_rw_01_egarch)))
print(DM_test(PH, PH_rw_01_egarch, PH_rw_01_figarch0633, len(PH_rw_01_egarch)))

dm_return(DM=-1.87479789346525, p_value=0.061194006515633)
dm_return(DM=-5.325880796333146, p_value=1.3152670917920156e-07)
dm_return(DM=-3.3960515274441936, p_value=0.0007182548661083916)


In [41]:
# IGARCH
print(DM_test(PH, PH_rw_01_igarch, PH_rw_01_figarch,  len(PH_rw_01_igarch)))
print(DM_test(PH, PH_rw_01_igarch, PH_rw_01_figarch0633, len(PH_rw_01_igarch)))

dm_return(DM=0.7177055637668838, p_value=0.47315374042889846)
dm_return(DM=0.2788339099663728, p_value=0.7804461977351672)


In [42]:
# FIGARCH
print(DM_test(PH, PH_rw_01_figarch, PH_rw_01_figarch0633, len(PH_rw_01_figarch)))

dm_return(DM=-1.4929417880729494, p_value=0.13585678935405882)


## 1.00 %

In [43]:
PH_rw_1_dn = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/DN/1%25/PH_VaR_RW-DN-1%25.csv')
PH_rw_1_hs = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/HS/1%25/PH_VaR_RW-HS-1%25.csv')
PH_rw_1_brw = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/BRW/1%25/PH_VaR_RW-BRW-1%25.csv')
PH_rw_1_garch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/GARCH/1%25/PH_VaR_RW-G-1%25.csv')
PH_rw_1_earch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EARCH/1%25/PH_VaR_RW-EA-1%25.csv')
PH_rw_1_egarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EGARCH/1%25/PH_VaR_RW-E-1%25.csv')
PH_rw_1_igarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/IGARCH/IGARCH-1%25-RW/PH_RW-I-1%25.csv')
PH_rw_1_figarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH/FIGARCH-1%25-RW/PH_RW-FI-1%25.csv')
PH_rw_1_figarch0633 = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH0.633/FIGARCH0.633-1%25-RW/PH_RW-FI0.633-1%25.csv')

In [44]:
# DN
print(DM_test(PH, PH_rw_1_dn, PH_rw_1_garch, len(PH_rw_1_dn)))
print(DM_test(PH, PH_rw_1_dn, PH_rw_1_earch, len(PH_rw_1_dn)))
print(DM_test(PH, PH_rw_1_dn, PH_rw_1_egarch, len(PH_rw_1_dn)))
print(DM_test(PH, PH_rw_1_dn, PH_rw_1_igarch, len(PH_rw_1_dn)))
print(DM_test(PH, PH_rw_1_dn, PH_rw_1_figarch, len(PH_rw_1_dn)))
print(DM_test(PH, PH_rw_1_dn, PH_rw_1_figarch0633, len(PH_rw_1_dn)))

dm_return(DM=-3.3294805983124487, p_value=0.0009109417488365572)
dm_return(DM=-0.38233386725227353, p_value=0.702317887101868)
dm_return(DM=-2.6659327968451367, p_value=0.007836412766563045)
dm_return(DM=-2.3425465046384066, p_value=0.019403176071196633)
dm_return(DM=-3.680427291571957, p_value=0.00024886556043103445)
dm_return(DM=-3.225124443840734, p_value=0.0013116407758769243)


In [45]:
# HS
print(DM_test(PH, PH_rw_1_hs, PH_rw_1_garch, len(PH_rw_1_hs)))
print(DM_test(PH, PH_rw_1_hs, PH_rw_1_earch, len(PH_rw_1_hs)))
print(DM_test(PH, PH_rw_1_hs, PH_rw_1_egarch, len(PH_rw_1_hs)))
print(DM_test(PH, PH_rw_1_hs, PH_rw_1_igarch, len(PH_rw_1_hs)))
print(DM_test(PH, PH_rw_1_hs, PH_rw_1_figarch, len(PH_rw_1_hs)))
print(DM_test(PH, PH_rw_1_hs, PH_rw_1_figarch0633, len(PH_rw_1_hs)))

dm_return(DM=-2.8991634816897864, p_value=0.00384662862500785)
dm_return(DM=1.1816642374736124, p_value=0.23769909038039333)
dm_return(DM=-2.0190847367599445, p_value=0.043820050773347545)
dm_return(DM=-2.123397865484204, p_value=0.03403475546191863)
dm_return(DM=-3.2435693347571877, p_value=0.0012306637976102245)
dm_return(DM=-2.9026122326818, p_value=0.003804980612718518)


In [46]:
# BRW
print(DM_test(PH, PH_rw_1_brw, PH_rw_1_garch, len(PH_rw_1_brw)))
print(DM_test(PH, PH_rw_1_brw, PH_rw_1_earch, len(PH_rw_1_brw)))
print(DM_test(PH, PH_rw_1_brw, PH_rw_1_egarch, len(PH_rw_1_brw)))
print(DM_test(PH, PH_rw_1_brw, PH_rw_1_igarch, len(PH_rw_1_brw)))
print(DM_test(PH, PH_rw_1_brw, PH_rw_1_figarch, len(PH_rw_1_brw)))
print(DM_test(PH, PH_rw_1_brw, PH_rw_1_figarch0633, len(PH_rw_1_brw)))

dm_return(DM=2.1571250641578676, p_value=0.03130073566371614)
dm_return(DM=12.38344121051578, p_value=2.784525925656304e-32)
dm_return(DM=5.356822261907947, p_value=1.1155662777779563e-07)
dm_return(DM=0.5005140655930355, p_value=0.6168543452019906)
dm_return(DM=1.888379963665377, p_value=0.059345809409055046)
dm_return(DM=0.9163007604211301, p_value=0.35979237340405956)


In [47]:
# GARCH
print(DM_test(PH, PH_rw_1_garch, PH_rw_1_earch, len(PH_rw_1_garch)))
print(DM_test(PH, PH_rw_1_garch, PH_rw_1_egarch, len(PH_rw_1_garch)))
print(DM_test(PH, PH_rw_1_garch, PH_rw_1_igarch, len(PH_rw_1_garch)))
print(DM_test(PH, PH_rw_1_garch, PH_rw_1_figarch, len(PH_rw_1_garch)))
print(DM_test(PH, PH_rw_1_garch, PH_rw_1_figarch0633, len(PH_rw_1_garch)))

dm_return(DM=3.4966393052711138, p_value=0.0004978003149934999)
dm_return(DM=4.411038422086805, p_value=1.1732869835563182e-05)
dm_return(DM=-0.9130093433170734, p_value=0.3615197390590622)
dm_return(DM=-1.640360869354039, p_value=0.10133333018680209)
dm_return(DM=-1.722636600137401, p_value=0.08535063798192989)


In [48]:
# EARCH
print(DM_test(PH, PH_rw_1_earch, PH_rw_1_egarch, len(PH_rw_1_earch)))
print(DM_test(PH, PH_rw_1_earch, PH_rw_1_igarch, len(PH_rw_1_earch)))
print(DM_test(PH, PH_rw_1_earch, PH_rw_1_figarch, len(PH_rw_1_earch)))
print(DM_test(PH, PH_rw_1_earch, PH_rw_1_figarch0633, len(PH_rw_1_earch)))

dm_return(DM=-2.7729418327860245, p_value=0.005687770648193646)
dm_return(DM=-2.491129521400004, p_value=0.012940606073603988)
dm_return(DM=-3.9511168435092374, p_value=8.483973724513977e-05)
dm_return(DM=-3.4807524733555, p_value=0.000527794321448095)


In [49]:
# EGARCH
print(DM_test(PH, PH_rw_1_egarch, PH_rw_1_igarch, len(PH_rw_1_egarch)))
print(DM_test(PH, PH_rw_1_egarch, PH_rw_1_figarch, len(PH_rw_1_egarch)))
print(DM_test(PH, PH_rw_1_egarch, PH_rw_1_figarch0633, len(PH_rw_1_egarch)))

dm_return(DM=-1.8969893040063328, p_value=0.05819849306579413)
dm_return(DM=-5.302461534046194, p_value=1.4890388905876727e-07)
dm_return(DM=-3.3884259816907187, p_value=0.000738224969782649)


In [50]:
# IGARCH
print(DM_test(PH, PH_rw_1_igarch, PH_rw_1_figarch,  len(PH_rw_1_igarch)))
print(DM_test(PH, PH_rw_1_igarch, PH_rw_1_figarch0633, len(PH_rw_1_igarch)))

dm_return(DM=0.7550598292074354, p_value=0.450441007376716)
dm_return(DM=0.3222254968136932, p_value=0.7473682127393764)


In [51]:
# FIGARCH
print(DM_test(PH, PH_rw_1_figarch, PH_rw_1_figarch0633, len(PH_rw_1_figarch)))

dm_return(DM=-1.5127700397222732, p_value=0.13074313287185158)


## 5.00 %

In [52]:
PH_rw_5_dn = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/DN/5%25/PH_VaR_RW-DN-5%25.csv')
PH_rw_5_hs = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/HS/5%25/PH_VaR_RW-HS-5%25.csv')
PH_rw_5_brw = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/BRW/5%25/PH_VaR_RW-BRW-5%25.csv')
PH_rw_5_garch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/GARCH/5%25/PH_VaR_RW-G-5%25.csv')
PH_rw_5_earch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EARCH/5%25/PH_VaR_RW-EA-5%25.csv')
PH_rw_5_egarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EGARCH/5%25/PH_VaR_RW-E-5%25.csv')
PH_rw_5_igarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/IGARCH/IGARCH-5%25-RW/PH_RW-I-5%25.csv')
PH_rw_5_figarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH/FIGARCH-5%25-RW/PH_RW-FI-5%25.csv')
PH_rw_5_figarch0633 = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH0.633/FIGARCH0.633-5%25-RW/PH_RW-FI0.633-5%25.csv')

In [53]:
# DN
print(DM_test(PH, PH_rw_5_dn, PH_rw_5_garch, len(PH_rw_5_dn)))
print(DM_test(PH, PH_rw_5_dn, PH_rw_5_earch, len(PH_rw_5_dn)))
print(DM_test(PH, PH_rw_5_dn, PH_rw_5_egarch, len(PH_rw_5_dn)))
print(DM_test(PH, PH_rw_5_dn, PH_rw_5_igarch, len(PH_rw_5_dn)))
print(DM_test(PH, PH_rw_5_dn, PH_rw_5_figarch, len(PH_rw_5_dn)))
print(DM_test(PH, PH_rw_5_dn, PH_rw_5_figarch0633, len(PH_rw_5_dn)))

dm_return(DM=-3.282730742876764, p_value=0.0010738440169782305)
dm_return(DM=-0.4097780002095011, p_value=0.6820813190696124)
dm_return(DM=-2.6126457415219035, p_value=0.009157385813644796)
dm_return(DM=-2.3339511815490215, p_value=0.0198513652070571)
dm_return(DM=-3.6319800867273644, p_value=0.0002996460098840184)
dm_return(DM=-3.186708296789771, p_value=0.0014963433670568914)


In [54]:
# HS
print(DM_test(PH, PH_rw_5_hs, PH_rw_5_garch, len(PH_rw_5_hs)))
print(DM_test(PH, PH_rw_5_hs, PH_rw_5_earch, len(PH_rw_5_hs)))
print(DM_test(PH, PH_rw_5_hs, PH_rw_5_egarch, len(PH_rw_5_hs)))
print(DM_test(PH, PH_rw_5_hs, PH_rw_5_igarch, len(PH_rw_5_hs)))
print(DM_test(PH, PH_rw_5_hs, PH_rw_5_figarch, len(PH_rw_5_hs)))
print(DM_test(PH, PH_rw_5_hs, PH_rw_5_figarch0633, len(PH_rw_5_hs)))

dm_return(DM=-5.254747153234015, p_value=1.914548574360885e-07)
dm_return(DM=-7.711905673457422, p_value=3.782373419211065e-14)
dm_return(DM=-5.56985089225483, p_value=3.5103135276903514e-08)
dm_return(DM=-3.346874489103318, p_value=0.0008564251691350124)
dm_return(DM=-5.611641089361874, p_value=2.7851365847348602e-08)
dm_return(DM=-4.659289631920557, p_value=3.7302824536239684e-06)


In [55]:
# BRW
print(DM_test(PH, PH_rw_5_brw, PH_rw_5_garch, len(PH_rw_5_brw)))
print(DM_test(PH, PH_rw_5_brw, PH_rw_5_earch, len(PH_rw_5_brw)))
print(DM_test(PH, PH_rw_5_brw, PH_rw_5_egarch, len(PH_rw_5_brw)))
print(DM_test(PH, PH_rw_5_brw, PH_rw_5_igarch, len(PH_rw_5_brw)))
print(DM_test(PH, PH_rw_5_brw, PH_rw_5_figarch, len(PH_rw_5_brw)))
print(DM_test(PH, PH_rw_5_brw, PH_rw_5_figarch0633, len(PH_rw_5_brw)))

dm_return(DM=-2.7536909383014354, p_value=0.006029848809530392)
dm_return(DM=1.2999571205496756, p_value=0.1939996806184827)
dm_return(DM=-1.8243343867591173, p_value=0.06848397148697559)
dm_return(DM=-2.0590418635411987, p_value=0.039821718393193734)
dm_return(DM=-3.117110193712938, p_value=0.0018933361194851935)
dm_return(DM=-2.7901084970434664, p_value=0.005397626527738508)


In [56]:
# GARCH
print(DM_test(PH, PH_rw_5_garch, PH_rw_5_earch, len(PH_rw_5_garch)))
print(DM_test(PH, PH_rw_5_garch, PH_rw_5_egarch, len(PH_rw_5_garch)))
print(DM_test(PH, PH_rw_5_garch, PH_rw_5_igarch, len(PH_rw_5_garch)))
print(DM_test(PH, PH_rw_5_garch, PH_rw_5_figarch, len(PH_rw_5_garch)))
print(DM_test(PH, PH_rw_5_garch, PH_rw_5_figarch0633, len(PH_rw_5_garch)))

dm_return(DM=3.4440825776540795, p_value=0.000603587071871898)
dm_return(DM=4.387191109299415, p_value=1.3060581407195144e-05)
dm_return(DM=-0.9620445122122372, p_value=0.3363253841432151)
dm_return(DM=-1.7459736539354986, p_value=0.08120922644339178)
dm_return(DM=-1.7688842237978455, p_value=0.07730395511918671)


In [57]:
# EARCH
print(DM_test(PH, PH_rw_5_earch, PH_rw_5_egarch, len(PH_rw_5_earch)))
print(DM_test(PH, PH_rw_5_earch, PH_rw_5_igarch, len(PH_rw_5_earch)))
print(DM_test(PH, PH_rw_5_earch, PH_rw_5_figarch, len(PH_rw_5_earch)))
print(DM_test(PH, PH_rw_5_earch, PH_rw_5_figarch0633, len(PH_rw_5_earch)))

dm_return(DM=-2.7085993554437127, p_value=0.006904882591165605)
dm_return(DM=-2.477827733480784, p_value=0.013429169472704758)
dm_return(DM=-3.8935427827382587, p_value=0.00010724755374196467)
dm_return(DM=-3.43335628514796, p_value=0.000627605977218283)


In [58]:
# EGARCH
print(DM_test(PH, PH_rw_5_egarch, PH_rw_5_igarch, len(PH_rw_5_egarch)))
print(DM_test(PH, PH_rw_5_egarch, PH_rw_5_figarch, len(PH_rw_5_egarch)))
print(DM_test(PH, PH_rw_5_egarch, PH_rw_5_figarch0633, len(PH_rw_5_egarch)))

dm_return(DM=-1.9183312996036144, p_value=0.05543378504538999)
dm_return(DM=-5.2673730598309545, p_value=1.791696737176649e-07)
dm_return(DM=-3.375464314032791, p_value=0.0007733593205742486)


In [59]:
# IGARCH
print(DM_test(PH, PH_rw_5_igarch, PH_rw_5_figarch,  len(PH_rw_5_igarch)))
print(DM_test(PH, PH_rw_5_igarch, PH_rw_5_figarch0633, len(PH_rw_5_igarch)))

dm_return(DM=0.7940750834656287, p_value=0.42739323625661385)
dm_return(DM=0.3686142625108827, p_value=0.712515287510231)


In [60]:
# FIGARCH
print(DM_test(PH, PH_rw_5_figarch, PH_rw_5_figarch0633, len(PH_rw_5_figarch)))

dm_return(DM=-1.5312990123302452, p_value=0.12610096310527197)
